<a href="https://colab.research.google.com/github/shashankdubey78/CAP5610_ML/blob/main/K_Means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import random
from multiprocessing import Pool
import numpy as np
import pandas as pd
import math
from math import sqrt

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def euclidean(x, y):
	try:
		return np.sqrt(np.sum(np.square(np.subtract(x, y))))
	except Exception as e:
		print(e)
		print(x)
		print(y)

def cosine(x, y):
	try:
		return 1 - (np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y)))
	except Exception as e:
		print(e)
		print(x)
		print(y)

def jaccard(x, y):
	try:
		return 1 - np.divide(np.sum(np.minimum(x, y)), np.sum(np.maximum(x, y)))
	except Exception as e:
		print(e)
		print(x)
		print(y)

def SSE(distance, instances, centroids):
	sse = 0
	for centroid in centroids:
		for instance in instances:
			sse += distance(instance, centroid) ** 2
	return sse

In [3]:
def assign(instance, centroids, distance):
    minDistance = float('Inf')
    minDistanceIndex = 0
    for i in range(0, len(centroids)):
        d = distance(instance, centroids[i])
        if (d < minDistance):
            minDistance = d
            minDistanceIndex = i
    return minDistanceIndex


def createEmptyListOfLists(numSubLists):
    myList = []
    for i in range(numSubLists):
        myList.append([])
    return myList


def assignAll(instances, centroids, distance):
    clusters = createEmptyListOfLists(len(centroids))
    assignments = np.empty(len(instances), dtype=np.int8)
    i = 0
    for instance in instances:
        clusterIndex = assign(instance, centroids, distance)
        clusters[clusterIndex].append(instance)
        assignments[i] = clusterIndex
        i += 1
    return (clusters, assignments)


def computeCentroids(clusters):
    centroids = []
    for i in range(len(clusters)):
        name = "centroid" + str(i)
        # centroid = meanInstance(name, clusters[i])
        centroid = np.mean(clusters[i], axis=0).tolist()
        centroids.append(centroid)
    return centroids


def kmeans(instances, k=10, distance=cosine, stopCondition='Max_Preset', iteration_limit=500, initCentroids=None):

    VALID_STOP_CONDITIONS = ['CENTROIDS_STABLE', 'SSE_INCREASED', 'MAX_PRESET', 'ANY']
    stopCondition = stopCondition.upper()
    if not stopCondition in VALID_STOP_CONDITIONS:
	    raise ValueError("Invalid stopCondition: %s" % stopCondition)

    starttime = time.time()

    if (initCentroids is None or len(initCentroids) < k):
        # randomly select k initial centroids
        rng = np.random.default_rng()
        centroids = rng.choice(
            instances, k, replace=False, shuffle=False).tolist()
    else:
        centroids = initCentroids
    prevCentroids = []
    iterations = 0
    # prevWithinss = withinss = float('inf')
    prevSSE = currentSSE = float('inf')
    keep_going = True
    stopReason = None
    while keep_going:
        iterations += 1
        (clusters, assignments) = assignAll(instances, centroids, distance)
        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        # prevWithinss = withinss

        if stopCondition == 'Centroids_Stable' or stopCondition == 'ANY':
            keep_going = (centroids != prevCentroids)
            stopReason = 'Centroids_Stable'
        if stopCondition == 'SSE_Increased' or stopCondition == 'ANY':
            prevSSE = currentSSE
            currentSSE = SSE(distance, instances, centroids)
            keep_going = (currentSSE < prevSSE)
            stopReason = 'SSE_Increased'
        if iterations >= iteration_limit:
            keep_going = False
            stopReason = 'Max_Preset'

    endtime = time.time()

    result = {
        '#iterations': iterations,
        'timeTaken': endtime - starttime,
        'SSE_': SSE(distance, instances, centroids),
        'assignments': assignments,
        'stoppingReason': stopReason,
        # 'clusters': clusters,
        # 'centroids': centroids,
    }
    return result


In [7]:
DATA_PATH = './data.csv'
LABEL_PATH = './label.csv'

DATA = pd.read_csv(DATA_PATH, header=None)
LABEL = pd.read_csv(LABEL_PATH, names=['truth'])

droplist = []
for column in DATA:
	if(DATA[column].max() == 0):
		droplist.append(column)
DATA.drop(columns=droplist, inplace=True)

SAMPLE_SIZE = 2000
DATA_SMALL = DATA.sample(SAMPLE_SIZE)
LABEL_SMALL = LABEL.iloc[DATA_SMALL.index]

DATA = DATA.to_numpy(dtype=np.int16)
# LABEL = LABEL.to_numpy(dtype=np.int16)
DATA_SMALL = DATA_SMALL.to_numpy(dtype=np.int16)
# LABEL_SMALL = LABEL_SMALL
CATEGORIES = len(np.unique(LABEL))

np.shape(DATA), np.shape(LABEL), np.shape(DATA_SMALL), np.shape(LABEL_SMALL)

((10000, 668), (10000, 1), (2000, 668), (2000, 1))

In [8]:
def runTests(argsObj, groundTruth):
	pool = Pool()
	results = pool.starmap(kmeans, argsObj.values())

	resultsObj = {}
	i = 0
	for key in argsObj:
		resultsObj[key] = results[i]
		i += 1

	benchmarks = pd.DataFrame(resultsObj).transpose()

	scores = pd.Series(dtype=float, name="accuracy")
	for key in argsObj.keys():
		assignment = benchmarks.assignments[key]
		scores[key] = score(assignment, groundTruth)

	benchmarks = pd.concat([benchmarks, scores], axis=1).drop('assignments', axis=1)
	return benchmarks

def score(assignments, groundTruth):
	clusterToLabel = {}
	# labelToCluster = {}
	clusterScores = pd.DataFrame()
	assignmentsSeries = pd.Series(assignments, index=groundTruth.index, name='assignments')
	assignmentTable = pd.concat([assignmentsSeries, groundTruth], axis=1)

	for i in range(CATEGORIES):
		count = assignmentTable.loc[assignmentTable['assignments'] == i].groupby('truth').count()
		count = count.assignments.rename(index=(i))
		clusterScores = pd.concat([clusterScores, count], axis=1)

	clusterScores.fillna(0, inplace=True)
	for cluster in clusterScores:
		assignedLabel = clusterScores[cluster].idxmax()
		clusterToLabel[cluster] = assignedLabel
		# labelToCluster[assignedLabel] = cluster
		clusterScores.drop(assignedLabel, inplace=True)

	assignedLabels = []
	for i in assignments:
		assignedLabels.append(clusterToLabel[i])

	return np.sum(assignedLabels == groundTruth.truth) / len(assignedLabels)

In [ ]:
args1 = {
    'Euclidean Dist.': [DATA, CATEGORIES, euclidean, 'Centroids_Stable'],
    'Cosine Dist.': [DATA, CATEGORIES, cosine, 'Centroids_Stable'],
    'Jaccard Dist.': [DATA, CATEGORIES, jaccard, 'Centroids_Stable'],
}

benchmarks1 = runTests(args1, LABEL)

In [ ]:
benchmarks1.style.background_gradient(axis=0, cmap ='gist_heat_r')

,#iterations,timeTaken,SSE_,stoppingReason,accuracy
Euclidean Dist.,52,331.426529,443356566640.472534,centroids,0.464900
Cosine Dist.,63,769.026565,23289.517435,centroids,0.476500
Jaccard Dist.,71,831.297473,55511.024249,centroids,0.492600


In [ ]:
args3 = {
    'Euclidean Dist.': [DATA, CATEGORIES, euclidean, 'any', 100],
    'Cosine Dist.': [DATA, CATEGORIES, cosine, 'any', 100],
    'Jaccard Dist.': [DATA, CATEGORIES, jaccard, 'any', 100],
}

benchmarks3 = runTests(args3, LABEL)

In [ ]:
benchmarks3.style.background_gradient(axis=0, cmap ='gist_heat_r')

,#iterations,timeTaken,SSE_,stoppingReason,accuracy
Euclidean Dist.,2,26.190858,434987691558.218262,SSE,0.345100
Cosine Dist.,2,49.499019,21048.990817,SSE,0.419500
Jaccard Dist.,6,142.822698,55316.236547,SSE,0.466800


In [ ]:
LIMIT = 100
args4 = {
    'Euclidean_Centroids': [DATA, CATEGORIES, euclidean, 'centroids'],
    'Euclidean_SSE_': [DATA, CATEGORIES, euclidean, 'SSE'],
    'Euclidean_Limit': [DATA, CATEGORIES, euclidean, 'limit', LIMIT],
    'Cosine_Centroids': [DATA, CATEGORIES, cosine, 'centroids'],
    'Cosine_SSE': [DATA, CATEGORIES, cosine, 'SSE'],
    'Cosine_Limit': [DATA, CATEGORIES, cosine, 'limit', LIMIT],
    'Jaccard_Centroids': [DATA, CATEGORIES, jaccard, 'centroids'],
    'Jaccard_SSE': [DATA, CATEGORIES, jaccard, 'SSE'],
    'Jaccard_Limit': [DATA, CATEGORIES, jaccard, 'limit', LIMIT],
}

benchmarks4 = runTests(args4, LABEL)

In [ ]:
benchmarks4.style.background_gradient(axis=0, cmap ='gist_heat_r')

,#iterations,timeTaken,SSE_,stoppingReason,accuracy
Euclidean_Centroids,105,686.112271,443458943452.834656,centroids,0.422500
Euclidean_SSE_,2,25.998306,437194728454.095215,SSE,0.388000
Euclidean_Limit,100,653.815143,434941966002.902893,limit,0.607400
Cosine_Centroids,50,616.963028,23446.471315,centroids,0.410600
Cosine_SSE,3,73.814700,21910.319488,SSE,0.481400
Cosine_Limit,100,1239.093836,23250.476178,limit,0.529400
Jaccard_Centroids,67,789.951151,55515.875871,centroids,0.500900
Jaccard_SSE,5,116.831634,55880.023714,SSE,0.427600
Jaccard_Limit,100,1190.735739,55504.742989,limit,0.523300
